In [ ]:
!pip install .

# Configuração

In [3]:
import dedupe
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm
from casamento_entidade.diversos import carregar_json, criar_dir
from casamento_entidade.entrada import df_para_dict
from casamento_entidade.modelos_ce import ModeloDedupe

pd.options.mode.chained_assignment = None

In [4]:
# Logs
arquivo_logging = 'saidas/execucao.log'
log_level = 'INFO'

import logging
logging.getLogger('dedupe').setLevel(logging.CRITICAL)
logging.getLogger('casamento_entidade').setLevel(logging.CRITICAL)

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename=arquivo_logging, level=log_level, filemode='w+')

In [5]:
# Arquivos de entrada/saida
arquivo_modelo_treino = Path(f'saidas/arquivo_treinamento.json')
arquivo_modelo_configuracoes = Path(f'saidas/arquivo_configuracao')

arquivo_dados_treino = 'dados_treino/dados_pessoa_treino_amostra.csv'

# Configuracoes dedupe
casamento_selecionado = 'proba'

tipos_casamento = {'exato': 'Exact', 'proba': 'String'}
campos_casamento = [
    {
        'field': 'num_cpf_cnpj',
        'type': tipos_casamento[casamento_selecionado],
        'has missing': True
    },
    {
        'field': 'nome_pessoa',
        'type': tipos_casamento[casamento_selecionado],
        'has missing': True
    },
    {
        'field': 'num_titulo_eleitor',
        'type': tipos_casamento[casamento_selecionado],
        'has missing': True
    },
    {
        'field': 'num_nis_nit',
        'type': tipos_casamento[casamento_selecionado],
        'has missing': True
    },
    {
        'field': 'num_cliente_cemig',
        'type': tipos_casamento[casamento_selecionado],
        'has missing': True
    },
    {
        'field': 'nome_pai',
        'type': tipos_casamento[casamento_selecionado],
        'has missing': True
    },
    {
        'field': 'nome_mae',
        'type': tipos_casamento[casamento_selecionado],
        'has missing': True
    },
    {
        'field': 'data_nasc',
        'type': tipos_casamento[casamento_selecionado],
        'has missing': True
    }
]

num_cores = 15
forcar_treinamento = False
sample_size = 15_000
blocked_proportion = 0.9
original_length = None

# Treinamento

Treinamento do Dedupe para **Casamento de Entidades** utilizando dados de treino.

## Entrada do Algoritmo

In [6]:
# Leitura
colunas_treino = ['uuid_pessoa', 'num_cpf_cnpj', 'nome_pessoa', 'data_nasc', 
                  'nome_mae', 'nome_pai', 'nit', 'nis', 'num_titulo_eleitor', 'num_cliente_cemig']
colunas_idx = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Colunas para ler do csv
df_treino = pd.read_csv(arquivo_dados_treino, low_memory=False, sep='#', header=None, usecols=colunas_idx)
df_treino.columns = colunas_treino

# Coluna Nis/Nit
df_treino['num_nis_nit'] = df_treino['nis'].combine_first(df_treino['nit'])
df_treino.drop(['nis', 'nit'], inplace=True, axis=1)

# Removendo uuid duplicados
df_treino.sort_values(by='num_cpf_cnpj', inplace=True, ignore_index=True)
df_treino.drop_duplicates(subset=['uuid_pessoa'], keep='first', inplace=True, 
                          ignore_index=True)

# UUID como indice
df_treino.set_index('uuid_pessoa', inplace=True)

# Conversao para str se probabilistico
if casamento_selecionado == 'proba': 
    for c in tqdm(df_treino.columns):
        df_treino = df_treino.astype(str)

# Entrada para dedupe
df_treino = df_treino.where(pd.notnull(df_treino), None)  # dedupe soh aceita None
df_treino.reset_index(drop=True, inplace=True)
dados_dedupe_treino = df_treino.to_dict(orient='index')  # dedupe soh aceita dict

## Treinamento com Aprendizado Ativo

In [ ]:
# Configurações Dedupe
if (arquivo_modelo_configuracoes.exists()) and (not forcar_treinamento):
    print(f'Carregando o modelo a partir do arquivo de configuração {arquivo_modelo_configuracoes}')
    with arquivo_modelo_configuracoes.open(mode='rb') as fp:
        modelo_ce = dedupe.StaticDedupe(settings_file=fp, num_cores=num_cores)
else:
    # Instanciação
    modelo_ce = dedupe.Dedupe(variable_definition=campos_casamento, num_cores=num_cores)

    # Preparando treino
    print('Preparando o treinamento do modelo...')
    if arquivo_modelo_treino.exists():
        print(f'\tLendo exemplos do arquivo de treinamento ({arquivo_modelo_treino}).')
        with arquivo_modelo_treino.open() as fp:
            modelo_ce.prepare_training(data=dados_dedupe_treino, training_file=fp,
                                       sample_size=sample_size, blocked_proportion=blocked_proportion,
                                       original_length=original_length)
    else:
        modelo_ce.prepare_training(data=dados_dedupe_treino, training_file=None,
                                   sample_size=sample_size, blocked_proportion=blocked_proportion,
                                   original_length=original_length)

    # Active Learning
    print('Iniciando o treinamento com Active Learning...')
    dedupe.console_label(modelo_ce)
    modelo_ce.train()

    # Salva configuração e exemplos de treino
    print('Salvando os arquivos de treino e configuração...')
    with arquivo_modelo_treino.open(mode='w+') as fp:
        modelo_ce.write_training(file_obj=fp)

    with arquivo_config.open(mode='wb+') as fp:
        modelo_ce.write_settings(file_obj=fp)

    # Liberando memória
    modelo_ce.cleanup_training()

Preparando o treinamento do modelo...
